<h1>Содержание<span class="tocSkip"></span></h1>
</font></a></span></li></ul></li><li><span><a href="#Загрузка-и-подготовка-данных" data-toc-modified-id="Загрузка-и-подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка и подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Импорт-библиотек" data-toc-modified-id="Импорт-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорт библиотек</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Очистка-текстов" data-toc-modified-id="Очистка-текстов-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Очистка текстов</a></span></li><li><span><a href="#Лемматизация-текстов" data-toc-modified-id="Лемматизация-текстов-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Лемматизация текстов</a></span></li><li><span><a href="#Определение-признаков-и-разделение-выборок" data-toc-modified-id="Определение-признаков-и-разделение-выборок-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>Определение признаков и разделение выборок</a></span></li><li><span><a href="#Подсчёт-величины-TF-IDF" data-toc-modified-id="Подсчёт-величины-TF-IDF-1.6"><span class="toc-item-num">1.6&nbsp;&nbsp;</span>Подсчёт величины TF-IDF</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Модель-логистической-регрессии" data-toc-modified-id="Модель-логистической-регрессии-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модель логистической регрессии</a></span></li><li><span><a href="#Модель-случайного-леса" data-toc-modified-id="Модель-случайного-леса-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель случайного леса</a></span></li><li><span><a href="#Проверка-лучшей-модели-на-тестовой-выборке" data-toc-modified-id="Проверка-лучшей-модели-на-тестовой-выборке-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Проверка лучшей модели на тестовой выборке</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Классификация-на-эмбеддингах-(BERT)" data-toc-modified-id="Классификация-на-эмбеддингах-(BERT)-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Классификация на эмбеддингах (BERT)</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»  c BERT

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Загрузка и подготовка данных

### Импорт библиотек

In [1]:
import sys
import os

import pandas as pd #импорт библиотеки pandas (работа с таблицами)
import numpy as np

import nltk #импорт библиотеки для работы с Natural Language
from nltk.corpus import stopwords as nltk_stopwords #импорт инструмента для определения стоп-слов

import spacy #импорт инструмента лемматизации

from tqdm import notebook #импорт инструмента процесса выполнения циклов

import re #импорт модуля для работы с регулярными выражениями(для очистки)

from sklearn.feature_extraction.text import TfidfVectorizer #импорт инструмента для расчёта TF-IDF
from sklearn.metrics import f1_score, make_scorer #импорт метрики f1, инструмента для интеграции метрики в gridsearch
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV #импорт инструментов выбора модели
from sklearn.ensemble import RandomForestClassifier #импорт модели случайного леса
from sklearn.linear_model import LogisticRegression #импорт модели логистической регрессии
from sklearn.pipeline import Pipeline

import torch
import transformers


### Загрузка данных

In [2]:
#путь для ревью
path1='/datasets/toxic_comments.csv'

#путь для работы
path2='toxic_comments.csv'

#выбор пути если он существует методом os
if os.path.exists(path1):
    #загрузка датасета методом pandas.read_csv
    df=pd.read_csv(path1,index_col=[0])
elif os.path.exists(path2):
    df=pd.read_csv(path2,index_col=[0])
else:
    print('Path does not exist')

In [3]:
#вывод информации о данных методом info()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [4]:
#вывод первых строк датасета
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
#исследование баланса классов
df.groupby('toxic').count()

,text
toxic,
0,143106
1,16186


Отсюда видно: в данных отсутствуют пропуски, тексты на английском языке, а также они не лемматизированы. И существует дисбаланс классов: нетоксичных комментариев почти на порядок больше токсичных.

### Очистка текстов

In [6]:
#создание списков текстов
corpus=list(df['text'])

In [7]:
#создадим функцию, которая оставит в тексте только англоязычные буквы
def clear_text(text):
    text_0=re.sub(r'[^a-zA-Z]',' ',text)
    text_1=text_0.split()
    text_2=" ".join(text_1)
    return text_2

In [8]:
#очистка текстов
for i in notebook.tqdm(range(len(corpus))):
    corpus[i]=clear_text(corpus[i])

  0%|          | 0/159292 [00:00<?, ?it/s]

In [9]:
#проверка очистки
print(df['text'][5])
print(corpus[5])

"

Congratulations from me as well, use the tools well.  · talk "
Congratulations from me as well use the tools well talk


### Лемматизация текстов

In [10]:
#загрузка инструмента лемматизации для английского языка
nlp=spacy.load('en_core_web_sm',disable=['parser','ner'])

In [11]:
#создадим фнукцию, которая вернёт лемматизированный текст
def lemmatize_text(text):
    doc=nlp(text)
    lemm_text=' '.join([token.lemma_ for token in doc])
    return lemm_text

In [12]:
#лемматизация текстов
for i in notebook.tqdm(range(len(corpus))):
    corpus[i]=lemmatize_text(corpus[i])

  0%|          | 0/159292 [00:00<?, ?it/s]

In [13]:
#проверка лемматизации
n=2
print(df['text'][n])
print(corpus[n])

Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits instead of my talk page. He seems to care more about the formatting than the actual info.
hey man I m really not try to edit war it s just that this guy be constantly remove relevant information and talk to I through edit instead of my talk page he seem to care more about the formatting than the actual info


### Определение признаков и разделение выборок

In [14]:
#задаём признаки и целевой признак
features=corpus
target=df['toxic']

In [15]:
#разделение выборок на тренировочную и тестовую
features_train,features_test,target_train,target_test=train_test_split(features,target,train_size=0.8)

### Подсчёт величины TF-IDF

In [16]:
#сохранение английских стоп-слов
stopwords=set(nltk_stopwords.words('english'))
#обозначение инструмента подсчёта tf-idf
count_tf_idf=TfidfVectorizer(stop_words=stopwords)

## Обучение

### Модель логистической регрессии

In [17]:
%%time
#обучение модели логистической регрессии с перебором гиперпараметров и взвешиванием классов
parametres={'clf__solver':['lbfgs','liblinear', 'newton-cg'],
            'clf__max_iter':[5000],
            'clf__class_weight':['balanced'],
            'clf__C':[1, 10]}
pipe_0=Pipeline([('tfidf',TfidfVectorizer(stop_words=stopwords)),
                 ('clf',LogisticRegression())])
best_model_0=GridSearchCV(estimator=pipe_0,
                          param_grid=parametres,
                          cv=4,
                          scoring=make_scorer(f1_score))
best_model_0.fit(features_train,target_train)

CPU times: user 10min 1s, sys: 9min 22s, total: 19min 24s
Wall time: 19min 28s


GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words={'a',
                                                                    'about',
                                                                    'above',
                                                                    'after',
                                                                    'again',
                                                                    'against',
                                                                    'ain',
                                                                    'all', 'am',
                                                                    'an', 'and',
                                                                    'any',
                                                                    'are',
                                                                    'aren',
         

In [18]:
#вывод результатов f1-меры для модели
print('f1-мера:',abs(best_model_0.best_score_))

f1-мера: 0.7670918490265854


### Модель случайного леса

In [20]:
%%time
#обучение модели случайного леса с перебором гиперпараметров
parametres={'clf__n_estimators': [2],
            'clf__max_depth': [750,1000],
            'clf__criterion':["gini"],
            'clf__min_samples_split':[4,6],
           'clf__min_samples_leaf':[1,3]}
pipe_1=Pipeline([('tfidf',TfidfVectorizer(stop_words=stopwords)),
                 ('clf',RandomForestClassifier(class_weight='balanced',random_state=12345))])
best_model_1=GridSearchCV(estimator=pipe_1,
                          param_grid=parametres,
                          cv=4,
                          scoring=make_scorer(f1_score))
best_model_1.fit(features_train,target_train)

CPU times: user 4min 48s, sys: 3.87 s, total: 4min 52s
Wall time: 4min 52s


GridSearchCV(cv=4,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(stop_words={'a',
                                                                    'about',
                                                                    'above',
                                                                    'after',
                                                                    'again',
                                                                    'against',
                                                                    'ain',
                                                                    'all', 'am',
                                                                    'an', 'and',
                                                                    'any',
                                                                    'are',
                                                                    'aren',
         

In [21]:
#вывод результатов f1-меры для модели
print('f1-мера:',abs(best_model_1.best_score_))

f1-мера: 0.4987316810086649


### Проверка лучшей модели на тестовой выборке

В ходе работы были обучены две модели с подбором гиперпараметров и кросс-валидацией: логистическая регрессия и случайный лес. Сравнительные результаты на кросс-валидации:

| Модель                 | f1-мера  | Время выполнения|
|------------------------|----------|-----------------|
| Логистическая регрессия| 0.77     | 19 мин 10 сек   |
| Случайный лес          | 0.52     | 4 мин 35 сек    |

Далее проверим модель Логистической регрессии (model_0) на тестовой выборке:

In [23]:
#предсказания на тестовой выборке
predictions_test=best_model_0.predict(features_test)
#вывод результатов f1-меры 
print('f1-мера:',f1_score(predictions_test,target_test))

f1-мера: 0.7641352549889135


## Выводы


    
В ходе работы были обучены две модели с подбором гиперпараметров и кросс-валидацией: логистическая регрессия и случайный лес. 

Логистическая регрессия обучалась в 4 раза дольше, чем случайный лес (20 минут против 4), но зато выдала удовлетворительный результат f1 меры больше 0.75, в отличие от случайного леса.

На тестовой же выборке наилучшая модель логистической регрессии показала значение f1-меры в 0.77.    
   

## Классификация на эмбеддингах (BERT)

Несмотря на то, что данный раздел предполагалось выполнять на локальной машине, с этим возникли пока непреодолимые трудности, так что я попытался сделать в Jupyter Hub, но с очень маленькой выборкой (на бОльших ядро умирает), чтобы хотя бы проверить рабочесть идеи.

In [26]:
#частичная выборка из датасета
df_bert=df[0:800]

In [27]:
#инициализация токенайзера
tokenizer=transformers.BertTokenizer.from_pretrained("unitary/toxic-bert")

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/174 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/811 [00:00<?, ?B/s]

In [28]:
#токенизация текстов с переводом в тензоры
tokenized=df_bert['text'].apply(lambda x: tokenizer.encode(x,add_special_tokens=True,truncation=True))

In [29]:
#определение максимальной длины тензора
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)        

In [30]:
#заполняем тензоры разной длины до наибольшей нулями
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])
#создаём маску для нулей
attention_mask = np.where(padded != 0, 1, 0)

In [31]:
#инициализация модели
model=transformers.BertModel.from_pretrained("unitary/toxic-bert")

Downloading:   0%|          | 0.00/418M [00:00<?, ?B/s]

Some weights of the model checkpoint at unitary/toxic-bert were not used when initializing BertModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [32]:
#обучение эмбеддингов
batch_size = 2
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())

  0%|          | 0/400 [00:00<?, ?it/s]

In [33]:
#задаём признаки и целевой признак
features = np.concatenate(embeddings)
target=df_bert['toxic']

In [34]:
#делим выборки
features_train,features_test,target_train,target_test=train_test_split(features,target,train_size=0.5)

In [35]:
#обучение модели логистической регрессии на эмбеддингах с взвешиванием классов
model=LogisticRegression(class_weight='balanced',random_state=12345)
model.fit(features_train,target_train)
predictions=model.predict(features_test)

In [36]:
f1_score(predictions,target_test)

0.9195402298850575

**Вывод**
    
Даже на такой маленькой выборке в 800 строк результат оказался лучше, чем для tf-idf: f1-score равняется 0.91. Однако и время обучения оказалось наибольшим из методов (порядка 40 минут), хотя в целом сопоставимым с суммарным временем лемматизации (примерно 15 минут) и обучением линейной регрессии (20 минут)
    


## Чек-лист проверки

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок (одна ошибка оставлена специально для проверки гипотезы о правильности понимания)
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Данные загружены и подготовлены
- [x]  Модели обучены
- [x]  Значение метрики *F1* не меньше 0.75
- [x]  Выводы написаны